# Grid point per wavelength calculation

`spyro` uses variable resolution element sizes meshes, in order to maximize the accuracy and minimize the computational costs. In this software, such elements are based on the acoustic wavelength, the Courant–Friedrichs–Lewy (CFL) condition and a mesh gradation rate. Stated that, the design of resolution becomes proportional to the wavelength of the acoustic wave. One assumption made is that all triangles are nearly equilateral, which is required for accurate simulation with FEMs.

The desired distribution of triangular edge lengths $l_e$ in the meshes are calculated using a ratio of the local seismic velocity (e.g., P-wavespeed) and the representative frequency of a source wavelet:
$$
l_e(\textbf{x}) \propto \frac{c(\textbf{x})}{C \cdot f_{source}}
$$
where $c(\textbf{x})$ is spatially variable P-wavespeed, $f_{source}$ is the representative frequency of a source wavelet and $C$ denotes the number of cells-per-wavelength:
$$
C \equiv \frac{\lambda}{l_e}
$$
The following figure is an example of a typical mesh size distribution for the synthetic P-wavespeed model Marmousi2.

<img src="marmousi_mesh.png" alt="Drawing" style="width: 600px;"/>

**Note:** mesh sizes must be smoothly varying (otherwise referred to as a graded) to avoid numerical errors when simulations are performed. `spyro` uses a mesh gradation rate of 15%, which was obtained through trial-and-error.

The parameters $C$ and $G$ (number of grid-point-per-wavelength) are related to one another through:
$$
G = \alpha(P) \cdot C
$$
where $\alpha(P)$ is a constant coefficient that is a function of the spatial polynomial degree $k$. Kong-Mulder-Veldhuizen (KMV) elements have a higher number of nodes-per-element, therefore they have a higher $\alpha$ per polynomial degree than standard Lagrange elements.

$\alpha(P)$ is calculated based on the square root of the number of DoF ($n_{DoF}$) per number of elements ($n_e$) in the mesh, 
$$
\sqrt{\frac{n_{DoF}}{n_e}}
$$

The selection of $C$, and consequently $G$, raise several important questions such as: what is the minimal $G$ that can minimize numerical dispersion error and how does the choice of $G$ affect the total DoF for a given problem.

These are important aspects as they yield significant effects on both the runtime and computational requirements of FWI with FEM.

## 1. Importing libraries

The required libraries for running the grid point function are:

In [1]:
from mpi4py import MPI
import numpy as np
from scipy import interpolate
import meshio
import SeismicMesh
import firedrake as fire
import time
import copy
import spyro

firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


## 2. Required functions

### 2.1. Minimum grid point calculator

`minimum_grid_point_calculator` is a function to calculate necessary grid point density.

**Parameters:**
- *grid_point_calculator_parameters*: Python 'dictionary'. It contains all parameters related to the experiment. An example is provided in the demo file.
    
**Returns:**
- *G*: `float`. Minimum grid point density necessary for a `experiment_type` mesh with a FEM whith 
    the degree and method specified within the specified error tolerance.

In [4]:
def minimum_grid_point_calculator(grid_point_calculator_parameters):
    G_reference = grid_point_calculator_parameters['G_reference']
    degree_reference = grid_point_calculator_parameters['reference_degree']
    G_initial = grid_point_calculator_parameters['G_initial']
    desired_degree = grid_point_calculator_parameters['desired_degree']
    TOL = grid_point_calculator_parameters['accepted_error_threshold']

    model = spyro.tools.create_model_for_grid_point_calculation(grid_point_calculator_parameters, degree_reference)
    
    comm = spyro.utils.mpi_init(model)
    
    print("Entering search", flush = True)
    p_exact = wave_solver(model, G =G_reference, comm = comm)
    print("p_exact calculation finished", flush = True)

    comm.comm.barrier()

    model = spyro.tools.create_model_for_grid_point_calculation(grid_point_calculator_parameters, desired_degree)
    G = searching_for_minimum(model, p_exact, TOL, starting_G=G_initial, comm = comm)

    return G

### 2.2. Wave solver

`wave_solver` is a function that reuns the standard model several times.

**Parameters:**
- *model*: Python 'dictionary'. Contains model options and parameters.
- *G*: `float`. Defined as above.
- *comm*: Firedrake.ensemble_communicator. The MPI communicator for parallelism.
    
**Returns:**
- *p_recv*: `float`. Recorded pressure values.

In [5]:
def wave_solver(model, G, comm = False):
    minimum_mesh_velocity = model['testing_parameters']['minimum_mesh_velocity']

    mesh = generate_mesh(model, G, comm)

    element = spyro.domains.space.FE_method(mesh, model["opts"]["method"], model["opts"]["degree"])
    V = fire.FunctionSpace(mesh, element)

    if model['testing_parameters']['experiment_type'] == 'homogeneous':
        vp_exact = fire.Constant(minimum_mesh_velocity)
    elif model['testing_parameters']['experiment_type'] == 'heterogeneous':
        vp_exact = spyro.io.interpolate(model, mesh, V, guess=False)

    if model["opts"]["method"] == 'KMV':
        estimate_max_eigenvalue=True
    else:
        estimate_max_eigenvalue=False
    new_dt = 0.2*spyro.estimate_timestep(mesh, V, vp_exact,estimate_max_eigenvalue=estimate_max_eigenvalue)

    model['timeaxis']['dt'] = comm.comm.allreduce(new_dt, op=MPI.MIN)
    if model['timeaxis']['dt'] > 0.001:
        model['timeaxis']['dt'] = 0.001
    if comm.comm.rank == 0:
        print(
            f"Maximum stable timestep is: {comm.comm.allreduce(new_dt, op=MPI.MIN)} seconds",
            flush=True,
        )
        print(
            f"Maximum stable timestep used is: {model['timeaxis']['dt']} seconds",
            flush=True,
        )

    sources = spyro.Sources(model, mesh, V, comm)
    receivers = spyro.Receivers(model, mesh, V, comm)
    wavelet = spyro.full_ricker_wavelet(
                                        dt=model["timeaxis"]["dt"],
                                        tf=model["timeaxis"]["tf"],
                                        freq=model["acquisition"]["frequency"],
                                    )

    for sn in range(model["acquisition"]["num_sources"]):
        if spyro.io.is_owner(comm, sn):
            t1 = time.time()
            p_field, p_recv = spyro.solvers.forward(
                model, mesh, comm, vp_exact, sources, wavelet, receivers, source_num=sn, output= False)
            print(time.time() - t1)

    return p_recv

### 2.3. Searching for minimum

`searching_for_minimum` is a function to search for the minimum `G` value. It starts from the accuracy value and runs a loop until the minimum is found.

**Parameters:**
- *p_exact*: defined as above.
- *TOL*: `float`. Tolerance...
- *accuracy*: `float`. .... Default value: 0.1
- *starting_G:* `float`. Initial guess value for the minimum grid point density. Default value: 7.0
    
**Returns:**
- *G*: `float`. Optimized value of `G`

In [7]:
def searching_for_minimum(model, p_exact, TOL, accuracy = 0.1, starting_G = 7.0, comm=False):
    error = 100.0
    G = starting_G

    # fast loop
    print("Entering fast loop", flush = True)
    while error > TOL:
        dif = max(G*0.1, accuracy)
        G = G + dif
        print('With G equal to '+str(G) )
        print("Entering wave solver", flush = True)
        p0 = wave_solver(model,G, comm)
        error = error_calc(p_exact, p0, model, comm = comm)
        print('Error of '+str(error))

    G -= dif
    G = np.round(G,1)-accuracy
    # slow loop
    if dif > accuracy :
        print("Entering slow loop", flush = True)
        error = 100.0
        while error > TOL:
            dif = accuracy
            G = G + dif
            print('With G equal to '+str(G) )
            print("Entering wave solver", flush = True)
            p0 = wave_solver(model,G, comm )
            error = error_calc(p_exact, p0, model, comm = comm)
            print('Error of '+str(error))

    return G

### 2.4. Grid point to mesh point converter for SeismicMesh

`grid_point_to_mesh_point_converter_for_seismicmesh` converts the `G` value obtained previously, which is related to the grid, to a mesh representation.

**Parameters:** All of them were defined previously
    
**Returns:**
- *M*: `float`. Minimum mesh point density necessary for a `experiment_type` mesh with a FEM whith the degree and method specified within the specified error tolerance

In [8]:
def grid_point_to_mesh_point_converter_for_seismicmesh(model, G):
    degree = model["opts"]['degree']
    if model["opts"]["method"] == 'KMV':
        if degree == 1:
            M = G/0.707813887967734
        if degree == 2:
            M = 0.5*G/0.8663141029672784
        if degree == 3:
            M = 0.2934695559090401*G/0.7483761673104953
        if degree == 4:
            M = 0.21132486540518713*G/0.7010127254535244
        if degree == 5:
            M = 0.20231237605867816*G/0.9381929803311276

    if model["opts"]["method"] == 'CG':
        raise ValueError("Correct M to G conversion to be inputed for CG")
        # if degree == 1:
        #     M = G
        # if degree == 2:
        #     M = 0.5*G
        # if degree == 3:
        #     M = 0.333333333333333*G
        # if degree == 4:
        #     M = 0.25*G
        # if degree == 5:
        #     M = 0.2*G

    if model["opts"]["method"] == 'spectral':
        raise ValueError("Correct M to G conversion to be inputed for spectral")
        # if degree == 1:
        #     M = G
        # if degree == 2:
        #     M = 0.5*G
        # if degree == 3:
        #     M = 0.27639320225002106*G
        # if degree == 4:
        #     M = 0.32732683535398854*G
        # if degree == 5:
        #     M = 0.23991190372440996*G

    return M

### 2.5. Error calculation

`error_calc` is associated with the fact that *p0* doesn't necessarily have the same *dt* as *p_exact*. Therefore, the missing points must be interpolated to have them at the same length.

**Parameters:** All of them were defined previously
    
**Returns:**
- *error*: `float`. 

In [9]:
def error_calc(p_exact, p, model, comm = False):
    
    # testing shape
    times_p_exact, r_p_exact = p_exact.shape
    times_p, r_p = p.shape
    if times_p_exact > times_p: #then we interpolate p_exact
        times, receivers = p.shape
        dt = model["timeaxis"]['tf']/times
        p_exact = time_interpolation(p_exact, p, model)
    elif times_p_exact < times_p: #then we interpolate p
        times, receivers = p_exact.shape
        dt = model["timeaxis"]['tf']/times
        p = time_interpolation(p, p_exact, model)
    else: #then we dont need to interpolate
        times, receivers = p.shape
        dt = model["timeaxis"]['tf']/times
    #p = time_interpolation(p, p_exact, model)

    p_diff = p_exact-p
    max_absolute_diff = 0.0
    max_percentage_diff = 0.0

    if comm.ensemble_comm.rank ==0:
        numerator = 0.0
        denominator = 0.0
        for receiver in range(receivers):
            numerator_time_int = 0.0
            denominator_time_int = 0.0
            for t in range(times-1):
                top_integration = (p_exact[t,receiver]-p[t,receiver])**2*dt
                bot_integration = (p_exact[t,receiver])**2*dt

                # Adding 1e-25 filter to receivers to eliminate noise
                numerator_time_int   += top_integration

                denominator_time_int += bot_integration


                diff = p_exact[t,receiver]-p[t,receiver]
                if abs(diff) > 1e-15 and abs(diff) > max_absolute_diff:
                    max_absolute_diff = copy.deepcopy(diff)
                
                if abs(diff) > 1e-15 and abs(p_exact[t,receiver]) > 1e-15:
                    percentage_diff = abs( diff/p_exact[t,receiver]  )*100
                    if percentage_diff > max_percentage_diff:
                        max_percentage_diff = copy.deepcopy(percentage_diff)

            numerator   += numerator_time_int
            denominator += denominator_time_int

    if denominator > 1e-15:
        error = np.sqrt(numerator/denominator)

    # if numerator < 1e-15:
    #     print('Warning: error too small to measure correctly.', flush = True)
    #     error = 0.0
    if denominator < 1e-15:
        print("Warning: receivers don't appear to register a shot.", flush = True)
        error = 0.0

    # print("ERROR IS ", flush = True)
    # print(error, flush = True)
    # print("Maximum absolute error ", flush = True)
    # print(max_absolute_diff, flush = True)
    # print("Maximum percentage error ", flush = True)
    # print(max_percentage_diff, flush = True)
    return error

## 3. Example

In [ ]:
from numbers import Real
import numpy as np
import math
import spyro

def tetrahedral_volume(p1, p2, p3, p4):
    (x1, y1, z1) = p1
    (x2, y2, z2) = p2
    (x3, y3, z3) = p3
    (x4, y4, z4) = p4

    A = np.array([x1, y1, z1])
    B = np.array([x2, y2, z2])
    C = np.array([x3, y3, z3])
    D = np.array([x4, y4, z4])

    volume = abs(1.0 / 6.0 * (np.dot(B - A, np.cross(C - A, D - A))))

    return volume

def triangle_area(p1, p2, p3):
    """Simple function to calculate triangle area based on its 3 vertices."""
    (x1, y1) = p1
    (x2, y2) = p2
    (x3, y3) = p3

    return abs(x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2)) / 2

def test_grid_calc2d():
    grid_point_calculator_parameters = {
        ## Experiment parameters
        'source_frequency' : 5.0, # Here we define the frequency of the Ricker wavelet source
        'minimum_velocity_in_the_domain' :  1.429, # The minimum velocity present in the domain.
        # if an homogeneous test case is used this velocity will be defined in the whole domain.
        'velocity_profile_type': 'homogeneous', # Either or heterogeneous. If heterogeneous is 
        #chosen be careful to have the desired velocity model below.
        'velocity_model_file_name': 'vel_z6.25m_x12.5m_exact.segy',
        'FEM_method_to_evaluate' : 'KMV', # FEM to evaluate such as `KMV` or `spectral` (GLL nodes on quads and hexas)
        'dimension' : 2, # Domain dimension. Either 2 or 3.
        'receiver_setup' : 'near', #Either near or line. Near defines a receiver grid near to the source,
        # line defines a line of point receivers with pre-established near and far offsets.

        ## Line search parameters
        'reference_degree': 4, # Degree to use in the reference case (int)
        'G_reference': 8.0, # grid point density to use in the reference case (float)
        'desired_degree': 4, # degree we are calculating G for. (int)
        'G_initial': 7.0, # Initial G for line search (float)
        'accepted_error_threshold': 0.07, 
        'g_accuracy': 1e-1
        }


    G = spyro.tools.minimum_grid_point_calculator(grid_point_calculator_parameters)
    inside = (6.9< G and G<8.0)
    print(G)
    assert inside

def test_input_models_receivers():
    test1 = True #testing if 2D receivers are inside the domain on an homogeneous case
    grid_point_calculator_parameters = {
        ## Experiment parameters
        'source_frequency' : 5.0, # Here we define the frequency of the Ricker wavelet source
        'minimum_velocity_in_the_domain' :  1.429, # The minimum velocity present in the domain.
        # if an homogeneous test case is used this velocity will be defined in the whole domain.
        'velocity_profile_type': 'homogeneous', # Either or heterogeneous. If heterogeneous is 
        #chosen be careful to have the desired velocity model below.
        'FEM_method_to_evaluate' : 'KMV', # FEM to evaluate such as `KMV` or `spectral` (GLL nodes on quads and hexas)
        'dimension' : 2, # Domain dimension. Either 2 or 3.
        'receiver_setup' : 'near', #Either near or line. Near defines a receiver grid near to the source,
        # line defines a line of point receivers with pre-established near and far offsets.

        ## Line search parameters
        'reference_degree': 4, # Degree to use in the reference case (int)
        'G_reference': 8.0, # grid point density to use in the reference case (float)
        'desired_degree': 4, # degree we are calculating G for. (int)
        'G_initial': 7.0, # Initial G for line search (float)
        'accepted_error_threshold': 0.05, 
        'g_accuracy': 1e-1
        }
    model = spyro.tools.create_model_for_grid_point_calculation(grid_point_calculator_parameters, 4)

    Lz = model["mesh"]['Lz']
    lz = model['BCs']['lz']
    Lx = model["mesh"]['Lx']
    lx = model['BCs']['lx']

    Real_Lz = Lz + lz
    Real_Lx = Lx + 2*lx

    p1 = (-Real_Lz,-lx)
    p2 = (-Real_Lz,Real_Lx-lx)
    p3 = (0.0,-lx)
    p4 = (0.0,Real_Lx-lx)

    areaSquare = Real_Lz*Real_Lx

    for r in model["acquisition"]["receiver_locations"]:
        area1 = triangle_area(p1, p2, r)
        area2 = triangle_area(p1, p3, r)
        area3 = triangle_area(p3, p4, r)
        area4 = triangle_area(p2, p4, r)
        test = math.isclose((area1 + area2 + area3 + area4), areaSquare, rel_tol=1e-09)
        if test == False:
            test1 = False

    test2 =True # For 3D case
    grid_point_calculator_parameters = {
        ## Experiment parameters
        'source_frequency' : 5.0, # Here we define the frequency of the Ricker wavelet source
        'minimum_velocity_in_the_domain' :  1.429, # The minimum velocity present in the domain.
        # if an homogeneous test case is used this velocity will be defined in the whole domain.
        'velocity_profile_type': 'homogeneous', # Either or heterogeneous. If heterogeneous is 
        #chosen be careful to have the desired velocity model below.
        'FEM_method_to_evaluate' : 'KMV', # FEM to evaluate such as `KMV` or `spectral` (GLL nodes on quads and hexas)
        'dimension' : 3, # Domain dimension. Either 2 or 3.
        'receiver_setup' : 'near', #Either near or line. Near defines a receiver grid near to the source,
        # line defines a line of point receivers with pre-established near and far offsets.

        ## Line search parameters
        'reference_degree': 4, # Degree to use in the reference case (int)
        'G_reference': 8.0, # grid point density to use in the reference case (float)
        'desired_degree': 4, # degree we are calculating G for. (int)
        'G_initial': 7.0, # Initial G for line search (float)
        'accepted_error_threshold': 0.05, 
        'g_accuracy': 1e-1
        }
    model = spyro.tools.create_model_for_grid_point_calculation(grid_point_calculator_parameters,4)

    # FInish volume test later
    # Lz = model["mesh"]['Lz']
    # lz = model['BCs']['lz']
    # Lx = model["mesh"]['Lx']
    # lx = model['BCs']['lx']
    # Ly = model["mesh"]['Ly']
    # ly= model['BCs']['ly']

    # Real_Lz = Lz + lz
    # Real_Lx = Lx + 2*lx
    # Real_Ly = Ly + 2*ly

    # p1 = (-Real_Lz, -lx       , -ly)
    # p2 = (-Real_Lz, -lx       , Real_Ly-ly)
    # p3 = (-Real_Lz, Real_Lx-lx, -ly)
    # p4 = (-Real_Lz, Real_Lx-lx, Real_Ly-ly)
    # p5 = (0.0     , -lx       , -ly)
    # p6 = (0.0     , -lx       , Real_Ly-ly)
    # p7 = (0.0     , Real_Lx-lx, -ly)
    # p8 = (0.0     , Real_Lx-lx, Real_Ly-ly)

    # volumeSquare = Real_Lx*Real_Ly*Real_Lz

    # for r in model["acquisition"]["receiver_locations"]:
    #     volume1 = tetrahedral_volume(p1, p2, r)
    #     volume2 = tetrahedral_volume(p1, p3, r)
    #     volume3 = tetrahedral_volume(p1, p4, r)
    #     volume4 = tetrahedral_volume(p1, p5, r)
    #     volume5 = tetrahedral_volume(p1, p6, r)
    #     volume6 = tetrahedral_volume(p1, p7, r)
    #     test = math.isclose((volume1 + volume2 + volume3 + volume4), volumeSquare, rel_tol=1e-09)
    #     if test == False:
    #         test1 = False


    assert all([test1, test2])


if __name__ == "__main__":
    test_grid_calc2d()

## 4. References

ROBERTS, K. J. et al. spyro: a firedrake-based wave propagation and full waveform
inversion finite element solver. Geoscientific Model Development Discussions, v. 2021, p.
1–47, 2021. Available in: <https://gmd.copernicus.org/preprints/gmd-2021-363/> 